In [22]:
import pandas as pd
import openpyxl
import awswrangler as awr

pegando excel

In [23]:
excel_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\historico_faturas.xlsx"

df_base = pd.read_excel(excel_path, engine='openpyxl')

gerando base de inadimplência de hoje

In [24]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_inadimplentes.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_inadimplentes = awr.athena.read_sql_query(query, database='silver')
df_inadimplentes = df_inadimplentes.drop_duplicates('ponteiro', keep='first')

df_inadimplentes.loc[:, 'data_atualizacao'] = df_inadimplentes['data_vencimento']
df_inadimplentes.loc[:, 'situacao']='INADIMPLENTE'


selecionando esse mês

In [25]:
today = pd.to_datetime('today').date()
year = today.year
month = today.month
df_inadimplentes['data_vencimento'] = pd.to_datetime(df_inadimplentes['data_vencimento'])


#df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento']>= pd.to_datetime('2025-01-31')]
df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.month == month]

df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.year == year]




tratando colunas df_inadimplentes

In [26]:
colunas_comuns = [
    "matricula",
    "conjunto",
    "cooperativa",
    "ponteiro",
    "numero_boleto",
    "nosso_numero",
    "unidade",
    "associado",
    "data_vencimento",
    "data_baixa",
    "valor_titulo",
    "valor_baixa",
    "data_emissao",
    "dias_atrasado",
    "data_atualizacao",
    "situacao"
]

for col in colunas_comuns:
    if col not in df_inadimplentes.columns:
        df_inadimplentes.loc[:, col] = pd.NA

df_inadimplentes = df_inadimplentes[colunas_comuns]

concatenando e removendo linhas inteiras duplicadas

In [27]:
df_inadimplentes_composto =  pd.concat([df_base, df_inadimplentes])
df_inadimplentes_composto = df_inadimplentes_composto.sort_values(by='data_vencimento', ascending=False)
df_inadimplentes_composto = df_inadimplentes_composto.drop_duplicates(keep='first')


filtrando apenas os ponteiros de inadimplentes do mês

In [28]:
df_inadimplentes_composto['data_vencimento'] = pd.to_datetime(df_inadimplentes_composto['data_vencimento'])
df_inadimplentes_composto_mes = df_inadimplentes_composto[df_inadimplentes_composto['data_vencimento'].dt.month == month]
df_inadimplentes_composto_mes = df_inadimplentes_composto[df_inadimplentes_composto['data_vencimento'].dt.year == year]

df_inadimplentes_lista = df_inadimplentes_composto_mes.loc[df_inadimplentes_composto_mes['data_vencimento'].notna() ,'ponteiro'].to_list()



gerando o relatório de pagamentos do mês e tratando colunas

In [29]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_baixadas.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_pagamentos = awr.athena.read_sql_query(query, database='silver')

colunas_comuns = [
    "matricula",
    "conjunto",
    "cooperativa",
    "ponteiro",
    "numero_boleto",
    "nosso_numero",
    "unidade",
    "associado",
    "data_vencimento",
    "data_baixa",
    "valor_titulo",
    "valor_baixa",
    "data_emissao",
    "dias_atrasado",
    "data_atualizacao",
    "situacao"
]
for col in colunas_comuns:
    if col not in df_pagamentos.columns:
        df_pagamentos.loc[:, col] = pd.NA
df_pagamentos = df_pagamentos[colunas_comuns]

tratando colunas

In [30]:
df_pagamentos.loc[:, 'data_atualizacao'] = df_pagamentos['data_baixa']
df_pagamentos.loc[:, 'situacao']='PAGO'

In [31]:


df_pagamentos['data_baixa'] = pd.to_datetime(df_pagamentos['data_baixa'])
df_pagamentos = df_pagamentos.drop_duplicates('ponteiro', keep='first')
#df_pagamentos = df_pagamentos[df_pagamentos['data_baixa']>= pd.to_datetime('2025-01-31').date()]
df_pagamentos = df_pagamentos[df_pagamentos['data_baixa'].dt.month == month]
df_pagamentos = df_pagamentos[df_pagamentos['data_baixa'].dt.year == year]

df_pagamentos_mensal = df_pagamentos[df_pagamentos['ponteiro'].isin(df_inadimplentes_lista)]


concatenando faturas baixadas dos vencidos deste mes com a base composta e atualizada

In [32]:
df_atualizado = pd.concat([df_inadimplentes_composto, df_pagamentos_mensal])
df_atualizado.drop_duplicates(subset=['matricula', 'conjunto', 'situacao'])

C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_26608\1517168924.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_atualizado = pd.concat([df_inadimplentes_composto, df_pagamentos_mensal])


,codigo_cadastro,associado,grupo,matricula,conjunto,aplicacao_financeira,data_emissao,data_vencimento,valor_titulo,codigo,...,status,id_set,parent,cooperativa,placa,chassi,situacao,data_atualizacao,data_baixa,valor_baixa
871,241.0,TRANSPORTADORA NOSSA SENHORA DO CARMO - EIRELI ME,RECEITAS,242,17392,Mensalidades/Benefícios,2025-06-06 00:00:00,2025-06-06 00:00:00,201.52,2256.0,...,ATIVO,17392.0,242.0,Viavante,QOX7I08,9BSR6X400J3935952,INADIMPLENTE,2025-06-06 00:00:00,NaT,NaN
2799,NaN,ANTONIO CARLOS BARRANCO GIMENES,NaN,34810,139320,NaN,2024-07-24,2025-06-06 00:00:00,1065.41,NaN,...,NaN,NaN,NaN,Segtruck,NaN,NaN,INADIMPLENTE,2025-06-06,NaT,NaN
703,1701.0,VIDROLOG TRANSPORTES LTDA,RECEITAS,1708,5893,Mensalidades/Benefícios,2024-11-27 00:00:00,2025-06-06 00:00:00,2006.65,1385.0,...,ATIVO,5893.0,1708.0,Viavante,OAP4C16,9BVAG20D9EE810731,INADIMPLENTE,2025-06-06 00:00:00,NaT,NaN
823,5434737.0,CLANDIO DO NASCIMENTO,RECEITAS,9994,17356,Primeira Parcela,2025-06-01 00:00:00,2025-06-06 00:00:00,284.25,53.0,...,ATIVO,17356.0,9994.0,Viavante,NYH5I00,9BVAS02C8AE763761,INADIMPLENTE,2025-06-06 00:00:00,NaT,NaN
984,6788.0,MARCELO MARCOS MARTINS,RECEITAS,7128,11375,Mensalidades/Benefícios,2025-03-05 00:00:00,2025-06-06 00:00:00,331.14,33.0,...,ATIVO,11375.0,7128.0,Viavante,NaN,NaN,INADIMPLENTE,2025-06-06 00:00:00,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1330,5417621.0,RODRIGO REZENDE DA SILVA EIRELI,RECEITAS,35156,140194,Adesões,2025-02-16 00:00:00,2025-02-25 00:00:00,350.00,24.0,...,ATIVO,140194.0,35156.0,Segtruck,NaN,NaN,INADIMPLENTE,2025-02-25 00:00:00,NaT,NaN
1182,1833.0,JOAO FILA TRANSPORTES LTDA,RECEITAS,1831,2313,Mensalidades/Benefícios,2025-02-20 00:00:00,2025-02-21 00:00:00,525.00,317.0,...,ATIVO,2313.0,1831.0,Viavante,QJI1E40,93ZM2SSH0J8830039,INADIMPLENTE,2025-02-21 00:00:00,NaT,NaN
277,4614.0,JOSE MARIO PIRES,RECEITAS,20360,10548,Mensalidades/Benefícios,2024-08-20 00:00:00,2025-02-20 00:00:00,225.71,233.0,...,ATIVO,10548.0,20360.0,Stcoop,OKD6D83,96BM934251KS049298,INADIMPLENTE,2025-02-20 00:00:00,NaT,NaN
589344,NaN,MJ POWER TRANSPORTES LTDA,NaN,1735,3725,NaN,NaN,2025-06-05,NaN,NaN,...,NaN,NaN,NaN,Viavante,NaN,NaN,PAGO,2025-06-05,2025-06-05,2655.28


ajustando formatação

In [33]:
colunas_comuns = [
    "matricula",
    "conjunto",
    "cooperativa",
    "ponteiro",
    "numero_boleto",
    "nosso_numero",
    "unidade",
    "associado",
    "data_vencimento",
    "data_baixa",
    "valor_titulo",
    "valor_baixa",
    "data_emissao",
    "dias_atrasado",
    "data_atualizacao",
    "situacao"
]
df_atualizado['data_vencimento'] = pd.to_datetime(df_atualizado['data_vencimento'])
df_atualizado['data_vencimento'] = df_atualizado['data_vencimento'].dt.date 

df_atualizado['data_baixa'] = pd.to_datetime(df_atualizado['data_baixa'])
df_atualizado['data_baixa'] = df_atualizado['data_baixa'].dt.date 

df_atualizado['data_atualizacao'] = pd.to_datetime(df_atualizado['data_atualizacao']).dt.date

retornando ao excel base atualizado

In [34]:
excel_save_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\historico_faturas.xlsx"
df_atualizado.to_excel(excel_save_path, engine='openpyxl', index=False, sheet_name='historico_faturas')